Distributed MPC for cooperation scheme using artificial references.

The goal is to implement formation control for a multi-agents system.

$m$ agents with dynamics $x_i(t+1) = f(x_i(t), u_i(t))$.


Implement a sequential scheme as outlined in the corresponding manuscript:

>M. Köhler, M. A. Müller, and F. Allgöwer, "Distributed MPC for Self-Organized Cooperation of Multi-Agent Systems,", 2023, available on arxiv. doi: 10.48550/arXiv.2210.10128


In [ ]:
import mkmpc
import numpy as np
import casadi as cas
import matplotlib.pyplot as plt

### Set up the system.
We use the 10 state quadcopter system taken from
>Hu, Haimin / Feng, Xuhui / Quirynen, Rien / Villanueva, Mario Eduardo / Houska, Boris 
>Real-Time Tube MPC Applied to a 10-State Quadrotor Model 
>2018
>2018 Annual American Control Conference (ACC) 

We use the Euler method to discretise the system.

In [ ]:
num_agents = 3 # Set number of agents.
state_dims = [10, 10, 10, 10, 10]  # Set the state dimensions.
input_dims = [3, 3, 3, 3, 3]  # Set the input dimensions.
output_dim = 3  # Set the output dimension, which is the same for all agents.

disc_step_size = 0.1  # Set step size for discretisation.
grav = 9.81  # Set the gravity of Earth.

# Specifiy initial states.
initial_state_list = [np.array([[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]),
                      np.array([[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]),
                      np.array([[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]),
                      np.array([[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]),
                      np.array([[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]])]

dynamics_list = []  # collect dynamic functions
output_maps = []  # collect output functions
for i in range(num_agents):
    x = cas.MX.sym('x', state_dims[i])  # define a symbolic state
    u = cas.MX.sym('u', input_dims[i])  # define a symbolic input
    
    dynamics_list.append(cas.Function('dynamics', [x,u],
                                      [cas.vertcat(x[0] + disc_step_size*x[3],
                                       x[1] + disc_step_size*x[4],
                                       x[2] + disc_step_size*x[5],
                                       x[3] + disc_step_size*grav*cas.tan(x[6]),
                                       x[4] + disc_step_size*grav*cas.tan(x[7]),
                                       x[5] + disc_step_size*(-grav + 0.91*u[2]),
                                       x[6] + disc_step_size*(-8*x[6] + x[8]),
                                       x[7] + disc_step_size*(-8*x[7] + x[9]),
                                       x[8] + disc_step_size*10*(-x[6] + u[0]),
                                       x[9] + disc_step_size*10*(-x[7] + u[1])
                                       )],
                                      ))  # define the dynamics
    output_maps.append(cas.Function('output', [x,u], [cas.vertcat(x[0], x[1], x[2])], ['x', 'u'], ['y']))  # first three states are the output
    
# Define the agents.
agents = []
for i in range(num_agents):
    # Initialise the agent.
    agents.append(mkmpc.agent(id = str(i+1), state_dim = state_dims[i], input_dim = input_dims[i],
                              dynamics = dynamics_list[i], initial_time=0, initial_state=initial_state_list[i],
                              output_map=output_maps[i], output_dim=output_dim))


Set the same constraints for each agent.

In [ ]:
# Set constraints for each agent.
box_state_constr = np.array([[-10, 10], [-10, 10], [-10, 10], [-10, 10], [-10, 10], [-10, 10], [-10, 10], [-10, 10], [-10, 10], [-10, 10]])

for agent in agents:
    agent.set_constraints(box_state_constraints=box_state_constr)  # set state constraints.
    agent.input_constraints["A"] = np.array([[1, 0, 0],
                                             [-1, 0, 0],
                                             [0, 1, 0],
                                             [0, -1, 0],
                                             [0, 0, 1],
                                             [0, 0, -1]])
    agent.input_constraints["b"] = np.array([[np.pi/9],
                                             [np.pi/9],
                                             [np.pi/9],
                                             [np.pi/9],
                                             [2*grav],
                                             [0]])

Set neighbours as
* $\mathcal{N}_1 = \{2, 5\}$
* $\mathcal{N}_2 = \{1, 3\}$
* $\mathcal{N}_3 = \{2, 1\}$

In [ ]:
# Set neighbours in lists as described above. Note that the index starts at 0.
agents[0].neighbours = [agents[1], agents[2]]
agents[1].neighbours = [agents[0], agents[2]]
agents[2].neighbours = [agents[1], agents[0]]

Define the stage cost for tracking, i.e. $l_i = \Vert x_i - x_{\mathrm{c},i} \Vert^2{Q_i} + \Vert u_i - u_{\mathrm{c},i} \Vert^2_{R_i}$.

In [ ]:
for i in range(num_agents):
    # Define the artificial equilibrium. 
    x = cas.MX.sym('x', state_dims[i])
    u = cas.MX.sym('u', input_dims[i])
    xc = cas.MX.sym('x_c', state_dims[i])
    uc = cas.MX.sym('u_c', input_dims[i])
    
    # Set the weight for the distance of the state to the equilibrium.
    Q = np.eye(state_dims[i])
    # Set the weight for the distance of the input to the equilibrium.
    R = np.eye(input_dims[i])
    
    stage_cost = cas.Function('stage_cost', [x, u, xc, uc], [ (x - xc).T@Q@(x - xc) + (u - uc).T@R@(u - uc) ],
                              ['x', 'u', 'xc', 'uc'], ['l'])
    
    # Add stage cost to agents.
    agents[i].stage_cost = stage_cost

### Define the cost for cooperation.


In [ ]:
yc1 = cas.MX.sym('yc1', output_dim)
yc2 = cas.MX.sym('yc2', output_dim)

dij = 1  # Define the distance each agent should have.
coop_weight = 1  # Define a weight for the cooperation cost.

# consensus and formation:
bilat_coop_cost = cas.Function('cooperation_cost', [yc1, yc2], [ coop_weight*( ( ((yc1[0:2] - yc2[0:2]).T@(yc1[0:2] - yc2[0:2])) - dij**2)**2 + (yc1[2] - yc2[2]).T@(yc1[2] - yc2[2]) )], ['yc1', 'yc2'], ['V_ij^c'])

# Set the bilateral cooperation cost for each agent.
for agent in agents:
    agent.bilat_coop_cost = bilat_coop_cost

Define an easy set of admissible cooperation outputs $\mathcal{Y}_i$, the same for all agents.

* $\mathcal{Y}_i = [-8, 8]^3 \cap \{y_i \mid y_{i,3} \ge 0\}$ $\forall i$

Define the sets in the form $A y_i^\mathrm{c} \le b$.

In [ ]:
# Set the A matrix of the constraint inequality which is the same for all agents.
for agent in agents:
    agent.cooperation_output_constraint = {}
    agent.cooperation_output_constraint["A"] = np.array([[1, 0, 0],
                                                         [-1, 0, 0],
                                                         [0, 1, 0],
                                                         [0, -1, 0],
                                                         [0, 0, 1],
                                                         [0, 0, -1]])
    agent.cooperation_output_constraint["b"] = np.array([[8],
                                                         [8],
                                                         [8],
                                                         [8],
                                                         [8],
                                                         [0]])


# Set up the sequential MPC scheme.

### Set simulation and MPC parameters.

In [ ]:
last_simulation_time = 600
horizon = 20
print('Horizon is', horizon*disc_step_size, 'seconds.')
print('Simulation time is', last_simulation_time*disc_step_size, 'seconds.')

### Initialisation of all agents:

In [ ]:
# Set all cooperation outputs to zero for the initialisation since 0 is an admissible cooperation output for all agents.
for agent in agents:
    agent.current_cooperation_output = np.zeros((agent.output_dim, 1))

# Set the initial state.
# agents[0].current_state = np.array([[0.00], [0.00], [1], [0], [0], [0], [0], [0], [0], [0]])
# agents[1].current_state = np.array([[0.00], [0.00], [2], [0], [0], [0], [0], [0], [0], [0]])
# agents[2].current_state = np.array([[0.00], [0.00], [3], [0], [0], [0], [0], [0], [0], [0]])

agents[0].current_state = np.array([[1e-5], [0.00], [1], [0], [0], [0], [0], [0], [0], [0]])
agents[1].current_state = np.array([[-1e-5], [1e-5], [2], [0], [0], [0], [0], [0], [0], [0]])
agents[2].current_state = np.array([[-1e-5], [-1e-5], [3], [0], [0], [0], [0], [0], [0], [0]])

# Take output given by initial state as first cooperation output.
for agent in agents:
    agent.current_cooperation_output = np.copy(agent.current_state[0:3])

# Solve an initial optimisation problem.
# This is done only to get a feel for the parameters, including the initial conditions, and has no effect on the actual scheme simulated below.
for agent in agents:
    warm_start = []
    # Add warm start of the input trajectory.
    for i in range(horizon):
        warm_start.append(np.array([[0], [0], [9.81/0.91]]))
    # Add warm start of the state trajectory.
    for i in range(horizon+1):
        warm_start.append(agent.current_state)
    # Add warm start of the cooperation input.
    warm_start.append(np.array([[0], [0], [9.81/0.91]]))
    # Add warm start of the cooperation state.
    warm_start.append(agent.current_state)
    # Add warm start of the cooperation output.
    warm_start.append(agent.current_state[0:3])
    warm_start = np.concatenate(warm_start)
    
    grad_size = 0
    saved_neighbours_cooperation_outputs = []
    for neighbour in agent.neighbours:
        #grad_size += np.linalg.norm(neighbour.current_cooperation_output[0:2] - agent.current_cooperation_output[0:2])
        grad_size += np.linalg.norm(neighbour.current_cooperation_output - agent.current_cooperation_output)
    if np.abs(grad_size) < 1e-8:
        # Perturb cooperation output of neighbours.
        for neighbour in agent.neighbours:
            print(neighbour.current_cooperation_output)
            # Save the cooperation output of the neighbour.
            saved_neighbours_cooperation_outputs.append(np.copy(neighbour.current_cooperation_output))
            # Perturb cooperation output of neighbour.
            if neighbour.id == '1':
                pass
                neighbour.current_cooperation_output = neighbour.current_cooperation_output + np.array([[0.01], [0], [0]])    
            elif neighbour.id == '2':
                pass
                neighbour.current_cooperation_output = neighbour.current_cooperation_output + np.array([[-0.01], [0.01], [0]])    
            elif neighbour.id == '3':
                pass
                neighbour.current_cooperation_output = neighbour.current_cooperation_output + np.array([[0.02], [-0.01], [0]])    
        # Generate the warm start and use it instead.
        warm_start_sol = mkmpc.MPC_for_cooperation(agent, horizon=horizon, warm_start=warm_start)
        warm_start = []
        # Append warm start of the input trajectoy by taking the old one, shifting it and appending the currently optimal equilibrium's input.
        for i in range(horizon-1):
                warm_start.append(warm_start_sol["u_opt"][0:agent.input_dim, i+1:i+2])
        warm_start.append(warm_start_sol["uc_opt"])
        # Append warm start of the state trajectory by taking the old one, shifting it and appending the currently optimal equilibrium's state.
        for i in range(horizon):
                warm_start.append(warm_start_sol["x_opt"][0:agent.state_dim, i+1:i+2])
        warm_start.append(warm_start_sol["xc_opt"])

        # Append warm start of cooperation input.
        warm_start.append(warm_start_sol["uc_opt"])
        # Append warm start of the cooperation state.
        warm_start.append(warm_start_sol["xc_opt"])
        
        # Append the warm start of the cooperation output.
        warm_start.append(warm_start_sol["yc_opt"])
        
        warm_start = np.concatenate(warm_start)
        
        # Restore cooperation outputs of the neighbours.
        for neighbour in agent.neighbours:
            neighbour.current_cooperation_output = np.copy(saved_neighbours_cooperation_outputs[agent.neighbours.index(neighbour)])

    agent.current_MPC_sol = mkmpc.MPC_for_cooperation(agent, horizon=horizon, warm_start=warm_start)
    agent.current_cooperation_output = np.copy(agent.current_MPC_sol["yc_opt"])

# Plot initial solution.
fig_init1, ax_init1 = plt.subplots()
fig_init2, ax_init2 = plt.subplots()
fig_init3, ax_init3 = plt.subplots()
for agent in agents:
    #fig, ax = plt.subplots()  # Uncomment to show individual plots.
    time_steps = range(0, horizon+1)
    ax_init1.plot(time_steps, agent.current_MPC_sol["x_opt"][0,:])
    ax_init1.scatter(horizon, agent.current_MPC_sol["yc_opt"][0], marker="x")
    ax_init1.grid(True)
    ax_init2.plot(time_steps, agent.current_MPC_sol["x_opt"][1,:])
    ax_init2.scatter(horizon, agent.current_MPC_sol["yc_opt"][1], marker="x")
    ax_init2.grid(True)
    ax_init3.plot(time_steps, agent.current_MPC_sol["x_opt"][2,:])
    ax_init3.scatter(horizon, agent.current_MPC_sol["yc_opt"][2], marker="x")
    ax_init3.grid(True)

# Take output given by initial state as first cooperation output.
for agent in agents:
    agent.current_cooperation_output = np.copy(agent.current_state[0:3])

### MPC algorithm

In [ ]:
# Keep track of the closed-loop system.
closed_loop_evolution = []

for t in range(last_simulation_time+1):
    # Track the time.
    closed_loop_evolution.append([None]*len(agents))
    
    # Go in sequence over the agents.
    for agent in agents:
        closed_loop_evolution[t][agents.index(agent)] = {"time":t}
        agent.current_time = t
        # Keep track of the current state.
        closed_loop_evolution[t][agents.index(agent)].update({"current_state":np.copy(agent.current_state)})
        
        # Generate a warm start. Decision vector: (u, x, uc, xc, yc)
        if t == 0:
            #warm_start = np.zeros((u.shape[0] + x.shape[0] + uc.shape[0] + xc.shape[0] + yc.shape[0], 1))
            warm_start = []
            # Add warm start of the input trajectory.
            for i in range(horizon):
                warm_start.append(np.array([[0], [0], [9.81/0.91]]))
            # Add warm start of the state trajectory.
            for i in range(horizon+1):
                warm_start.append(agent.current_state)
            # Add warm start of the cooperation input.
            warm_start.append(np.array([[0], [0], [9.81/0.91]]))
            # Add warm start of the cooperation state.
            warm_start.append(agent.current_state)
            # Add warm start of the cooperation output.
            warm_start.append(agent.current_state[0:3])
            warm_start = np.concatenate(warm_start)
        else:
            warm_start = []
            # Append warm start of the input trajectoy by taking the old one, shifting it and appending the currently optimal equilibrium's input.
            for i in range(horizon-1):
                 warm_start.append(agent.current_MPC_sol["u_opt"][0:agent.input_dim, i+1:i+2])
            warm_start.append(agent.current_MPC_sol["uc_opt"])
            # Append warm start of the state trajectory by taking the old one, shifting it and appending the currently optimal equilibrium's state.
            for i in range(horizon):
                 warm_start.append(agent.current_MPC_sol["x_opt"][0:agent.state_dim, i+1:i+2])
            warm_start.append(agent.current_MPC_sol["xc_opt"])

            # Append warm start of cooperation input.
            warm_start.append(agent.current_MPC_sol["uc_opt"])
            # Append warm start of the cooperation state.
            warm_start.append(agent.current_MPC_sol["xc_opt"])
            
            # Append the warm start of the cooperation output.
            warm_start.append(agent.current_MPC_sol["yc_opt"])
            
            warm_start = np.concatenate(warm_start)
            
        # Check if gradient w.r.t. formation goal is zero, e.g. all agents are at the same position.
        # If the gradient is zero, perturb the position of the neighbour to provide a suitable warm start for the solver.
        # Note that this is unlikely to arrise in practise expect if the initial condition is chosen badly as done in the paper (on purpose).
        grad_size_approx = 0
        saved_neighbours_cooperation_outputs = []
        for neighbour in agent.neighbours:
            grad_size_approx += np.linalg.norm(neighbour.current_cooperation_output[0:2] - agent.current_cooperation_output[0:2])
        if np.abs(grad_size_approx) < 1e-8:
            # Perturb cooperation output of neighbours.
            for neighbour in agent.neighbours:
                neighbours_perturbed = 1 # Set a flag to later restore the cooperation outputs of the neighbours.
                # Save the cooperation output of the neighbour.
                saved_neighbours_cooperation_outputs.append(np.copy(neighbour.current_cooperation_output))
                # Perturb cooperation output of neighbour.
                if neighbour.id == '1':
                    neighbour.current_cooperation_output = neighbour.current_cooperation_output + np.array([[0.01], [0], [0]])    
                elif neighbour.id == '2':
                    neighbour.current_cooperation_output = neighbour.current_cooperation_output + np.array([[-0.01], [0.01], [0]])    
                elif neighbour.id == '3':
                    neighbour.current_cooperation_output = neighbour.current_cooperation_output + np.array([[0.02], [-0.01], [0]])    
            # Generate the warm start and use it instead.
            warm_start_sol = mkmpc.MPC_for_cooperation(agent, horizon=horizon, warm_start=warm_start)
            warm_start = []
            # Append warm start of the input trajectoy by taking the old one, shifting it and appending the currently optimal equilibrium's input.
            for i in range(horizon-1):
                 warm_start.append(warm_start_sol["u_opt"][0:agent.input_dim, i+1:i+2])
            warm_start.append(warm_start_sol["uc_opt"])
            # Append warm start of the state trajectory by taking the old one, shifting it and appending the currently optimal equilibrium's state.
            for i in range(horizon):
                 warm_start.append(warm_start_sol["x_opt"][0:agent.state_dim, i+1:i+2])
            warm_start.append(warm_start_sol["xc_opt"])

            # Append warm start of cooperation input.
            warm_start.append(warm_start_sol["uc_opt"])
            # Append warm start of the cooperation state.
            warm_start.append(warm_start_sol["xc_opt"])
            
            # Append the warm start of the cooperation output.
            warm_start.append(warm_start_sol["yc_opt"])
            
            warm_start = np.concatenate(warm_start)
            
            # Restore cooperation outputs of the neighbours.
            for neighbour in agent.neighbours:
                neighbour.current_cooperation_output = np.copy(saved_neighbours_cooperation_outputs[agent.neighbours.index(neighbour)])
        
        # Solve the MPC problem.
        agent.current_MPC_sol = mkmpc.MPC_for_cooperation(agent, horizon=horizon, warm_start=warm_start)
        # Keep track of the solution.
        closed_loop_evolution[t][agents.index(agent)].update(agent.current_MPC_sol)
        # Update the current state of the agent. 
        # Without model errors and with satisfaction of the dynamic constraint, the next predicted state will be the next closed-loop state.
        agent.current_state = np.copy(agent.current_MPC_sol["x_opt"][0:agent.state_dim, 1:2])
        # Update the cooperation output of the agent.
        agent.current_cooperation_output = np.copy(agent.current_MPC_sol["yc_opt"])

# Plots
#### Plot the closed-loop evolution.
Note that these plots are not the plots used in the paper, which where created using a different work flow.

In [ ]:
# Extract the closed-loop evolution.
time_steps = range(0, last_simulation_time+1)

# Create a vector with time in seconds.
time = []
for t in time_steps:
    time.append(t*disc_step_size)
    
fig1_states, (ax_states_1, ax_states_2, ax_states_3) = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(20,6))
fig2_states, (ax_states_4, ax_states_5, ax_states_6) = plt.subplots(nrows=1, ncols=3, sharex=True, figsize=(20,6))
fig3_states, (ax_states_7, ax_states_8, ax_states_9, ax_states_10) = plt.subplots(nrows=1, ncols=4, sharex=True, figsize=(20,6))

for ax_i in fig1_states.axes:
    ax_i.grid(True)
for ax_i in fig2_states.axes:
    ax_i.grid(True)
for ax_i in fig3_states.axes:
    ax_i.grid(True)
    
fig2, ax2 = plt.subplots()
ax2.grid(True)
ax3 = plt.figure().add_subplot(projection='3d')
ax3.grid(True)

for agent in agents:
    # Extract state evolution of agent.
    agent_state_evo = []
    for t in time_steps:
        agent_state_evo.append(closed_loop_evolution[t][agents.index(agent)]["current_state"])
    # Build state evolution matrix.
    state_evo_mat = np.concatenate(agent_state_evo, axis=1)
    
    # Extract input evolution of agent.
    agent_input_evo = []
    for t in time_steps:
        agent_input_evo.append(closed_loop_evolution[t][agents.index(agent)]["u_opt"][0:agent.input_dim, 0:1])
    input_evo_mat = np.concatenate(agent_input_evo, axis=1)
    
    label_str = "Agent " + agent.id  # For labelling the plot.
    
    ax_states_1.plot(time, state_evo_mat[0, :], label=label_str, marker='x', markersize=0)
    ax_states_1.set_ylabel('pos1')
    ax_states_1.set_xlabel('time in s')
    
    ax_states_2.plot(time, state_evo_mat[1, :], label=label_str, marker='x', markersize=0)
    ax_states_2.set_ylabel('pos2')
    ax_states_2.set_xlabel('time in s')
    
    # Plot evolution of the altitute.
    ax_states_3.plot(time, state_evo_mat[2, :], label=label_str, marker='x', markersize=0)
    ax_states_3.set_ylabel('altitute in m')
    ax_states_3.set_xlabel('time in s')
    
    ax_states_4.plot(time, state_evo_mat[3, :], label=label_str, marker='x', markersize=0)
    ax_states_4.set_ylabel('x_4')
    ax_states_4.set_xlabel('time in seconds')
    
    ax_states_5.plot(time, state_evo_mat[4, :], label=label_str, marker='x', markersize=0)
    ax_states_5.set_ylabel('x_5')
    ax_states_5.set_xlabel('time in seconds')
    
    ax_states_6.plot(time, state_evo_mat[5, :], label=label_str, marker='x', markersize=0)
    ax_states_6.set_ylabel('x_6')
    ax_states_6.set_xlabel('time in seconds')
    
    ax_states_7.plot(time, state_evo_mat[6, :], label=label_str, marker='x', markersize=0)
    ax_states_7.set_ylabel('x_7')
    ax_states_7.set_xlabel('time in seconds')
    
    ax_states_8.plot(time, state_evo_mat[7, :], label=label_str, marker='x', markersize=0)
    ax_states_8.set_ylabel('x_8')
    ax_states_8.set_xlabel('time in seconds')
    
    ax_states_9.plot(time, state_evo_mat[8, :], label=label_str, marker='x', markersize=0)
    ax_states_9.set_ylabel('x_9')
    ax_states_9.set_xlabel('time in seconds')
    
    ax_states_10.plot(time, state_evo_mat[9, :], label=label_str, marker='x', markersize=0)
    ax_states_10.set_ylabel('x_10')
    ax_states_10.set_xlabel('time in seconds')
    
    # Plot 2D evolution of position without altitute.
    ax2.plot(state_evo_mat[0, :], state_evo_mat[1, :], label=label_str, marker='x', markersize=0)
    
    # Plot 3D evolution of position.
    ax3.plot(state_evo_mat[0, :], state_evo_mat[1, :], state_evo_mat[2, :])